# WEB SCRAPING-ASSIGNMENT3

In [1]:
# calling the required libraries for the entire assignment questions
from bs4 import BeautifulSoup   
import requests
import selenium
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.common.by import By
import time
import re
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.support import expected_conditions as EC
import warnings
warnings.filterwarnings("ignore")

# QUESTION 1

In [2]:
# 1. Write a python program which searches all the products under a particular product from www.amazon.in. 
# The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

# Connect to the driver
driver=webdriver.Chrome(r"chromedriver.exe")

# Opening amazon page on automated chrome web browser
driver.get("https://www.amazon.in/")
time.sleep(3)  # 3 seconds sleep

# maximize the window's size
driver.maximize_window()

# entering search key as required in the question
product=driver.find_element(By.ID, "twotabsearchtextbox")  # finding search box
product.send_keys(input("Enter the product name: "))  # Giving the product name as input

# clicking on search button
search=driver.find_element(By.ID, "nav-search-submit-button")  # finding search button
search.click()
driver.implicitly_wait(3) # wait the driver for 3 seconds

Enter the product name: guitar


# QUESTION 2

In [3]:
# In the above question, now scrape the following details of each product listed in first 3 pages of your search results 
# and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the 
# products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", 
# "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any 
# of the product then replace it by “-“.

# scrap all product urls
product_urls=[]
start=0
end=3
for page in range(start,end): # for loop for scrapping 3 pages
    # getting all urls of products displayed on amazon page
    url=driver.find_elements(By.XPATH,'//a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]')
    for i in url:  # for loop getting the data from urls 
        product_urls.append(i.get_attribute("href"))  # getting urls through href attribute  
    # finding next button and click on it to go to next page    
    next_button=driver.find_element(By.XPATH,'//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')
    next_button.click()
    driver.implicitly_wait(2) # wait the driver for 2 seconds

# scrapping details from each url of products
Brand_Name=[]
Name_of_the_product=[]
Price=[]
Return_Exchange=[]
Expected_Delivery=[]
Availability=[]

for i in product_urls: # loop for every product in the list
    driver.get(i)  # Opening each url 
    driver.implicitly_wait(2) # wait the driver for 2 seconds
    #getting brand name details
    try: 
        brand=driver.find_element(By.XPATH,'//div[@id="productOverview_feature_div"]/div/table/tbody/tr[1]/td[2]/span')
        Brand_Name.append(brand.text)
    except NoSuchElementException:   # exception handling for brand name
        Brand_Name.append("_")       # replace with "_" for missing details
        
    # getting name of the product details
    try:        
        product_name=driver.find_element(By.XPATH,'//div[@id="title_feature_div"]/div/h1/span')
        Name_of_the_product.append(product_name.text)
    except NoSuchElementException:
        Name_of_the_product.append("_")
    
    # getting price details
    try:
        price_data=driver.find_element(By.XPATH,'//div[@id="corePriceDisplay_desktop_feature_div"]/div//span[@class="a-price-whole"]')
        Price.append(price_data.text)
    except NoSuchElementException:
        Price.append("_")
        
    #getting return/exchange details
    try:
        Return=driver.find_element(By.XPATH,'//span[@class="a-declarative"]/div[2]/a')
        Return_Exchange.append(Return.text)
    except NoSuchElementException:
        Return_Exchange.append("_")
        
    # scrapping expected delivery details
    try:
        delivery=driver.find_element(By.XPATH,'//div[@id="mir-layout-DELIVERY_BLOCK"]/div[2]/span/span')
        Expected_Delivery.append(delivery.text)
    except NoSuchElementException:
        Expected_Delivery.append("_")
        
    # scrapping availability details
    try:
        stock_availability=driver.find_element(By.XPATH,'//div[@id="availability"]/span')
        Availability.append(stock_availability.text)
    except NoSuchElementException:
        Availability.append("_")
        
# printing lengths of all scrapped items
print(len(Brand_Name),len(Name_of_the_product),len(Price),len(Return_Exchange),len(Expected_Delivery),len(Availability),len(product_urls)) # printing the length of the list items
        
driver.close()  # closing the driver

#creating dataframe with the data as required in the question
df=pd.DataFrame({'Brand Name':Brand_Name,'Name of the Product':Name_of_the_product,'Price':Price,'Return/Exchange':Return_Exchange,'Expected Delivery':Expected_Delivery,'Availability':Availability,'Product URL':product_urls})

# writing the data to csv file
df.to_csv('Assignment3-Question2-file.csv')

# displaying the dataframe
df

92 92 92 92 92 92 92


,Brand Name,Name of the Product,Price,Return/Exchange,Expected Delivery,Availability,Product URL
0,Kadence,Kadence Slowhand Premium Jumbo Semi Acoustic G...,"9,900",7 days Replacement,"Wednesday, 28 December",,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,Kadence,Kadence Frontier guitar with Online Guitar lea...,"4,949",7 days Replacement,"Tuesday, 27 December",,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,Multi-coloured,Hontral S4 String Decor Guitar Children's Musi...,899,7 days Replacement,"Saturday, 31 December",In stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
3,Gibz's,Gibz's G-38 Matte Acoustic Guitar With Truss R...,"4,498",7 days Replacement,"Thursday, 29 December",Only 2 left in stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
4,Juârez,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...","2,149",7 days Replacement,"Thursday, 29 December",In stock.,https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...,...,...,...
87,ORANGEWOOD GUITARS,ORANGEWOOD REY-S SPRUCE GRAND AUDITORIUM CUTAW...,"13,930",7 days Replacement,"Thursday, 29 December",Only 2 left in stock.,https://www.amazon.in/Orangewood-Auditorium-Ac...
88,Kadence,"Kadence Frontier 34inch guitar for Kids, Trave...","3,598",7 days Replacement,"Saturday, 31 December",In stock.,https://www.amazon.in/Kadence-Professional-Gui...
89,Rosewood,Takamine GD30CE-12NAT Dreadnought 12-String Cu...,"1,00,769",7 days Replacement,"Saturday, 31 December",Only 1 left in stock.,https://www.amazon.in/Takamine-GD30CE-12NAT-Dr...
90,Walden,"Walden, Acoustic Guitar, Grand Auditorium with...","42,555",7 days Replacement,"Wednesday, 28 December",Only 1 left in stock (more on the way).,https://www.amazon.in/Walden-Acoustic-Auditori...


# QUESTION 3

In [5]:
# Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords
# ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

# Connect to the driver
driver=webdriver.Chrome(r"chromedriver.exe")

# Opening required web page on automated chrome web browser
driver.get("https://images.google.com/")
time.sleep(2) # wait the driver for 2 seconds

# maximize the window's size
driver.maximize_window()

# defining the list with all search key words as required in the question
key_words=["fruits","cars","Machine Learning","Guitar","Cakes"]

for kw in key_words:   # for loop on all key_words list
    
    if key_words.index(kw)==0:  # Checking if key_words index=0 or not.
        # finding keyword as required in the question
        product=driver.find_element(By.XPATH,'//div[@class="RNNXgb"]/div/div[2]/input')
        product.send_keys(kw)  # give the key word in search location
        
        # clicking on search button
        search=driver.find_element(By.XPATH,'//button[@jsname="Tg7LZd"]')
        search.click()
        driver.implicitly_wait(3) # seconds
        
    if key_words.index(kw)!=0:  # Checking if key_words index is not equal to 0 or not. Search element is different for index>0
        # finding keyword as required in the question
        product=driver.find_element(By.XPATH,'//div[@class="dAEiw dXdtIf"]/div[1]/form/div[1]/div[2]/div/div[2]/input')
        # clear the search field before entering next key word
        product.clear()
        product.send_keys(kw)  # give key word in search location 
        
        # finding and clicking on search button
        search=driver.find_element(By.XPATH,'//button[@class="rCGXm"]')
        search.click()
        driver.implicitly_wait(3) # seconds

    img_data = []

    for _ in range(20):  # scroll the page 20 times
        driver.execute_script("window.scrollBy(0,500)")   # scrolling the page to load 500 pixels

    images=driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')  # finding all the image elements

    img_urls=[]
    for i in images: # looping for all image elements
        source=i.get_attribute('src')  # get image url from image element
        if source is not None:  # checking if image url exists or not
            if(source[0:4]=='http'):   # checking if first 4 characters of url matches with "http" or not
                img_urls.append(source) # appending urls in the list
            
    for i in range(len(img_urls)): # looping for all image urls
        if i>10:  # checking if images count is >10
            break  # break the loop after getting 10 images for each key word
        print("Downloading {0} of {1} images of {2}".format(i,10,kw))  # print the line while extracting images
        response=requests.get(img_urls[i])  # checking image url response code using requests
        # opening file in local directory using file name extension with search key word name
        file=open(r"C:\Users\Admin\Desktop\Assignment\Images\image_"+str(kw)+str(i)+".jpg","wb") 
        file.write(response.content) # saving the images
        
driver.close()   # closing the driver

# QUESTION 4

In [8]:
# Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and 
# scrape following details for all the search results displayed on 1st page. Details to be scraped: 
# “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, 
# “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. 
# Save your results in a dataframe and CSV.

# Connect to the driver
driver=webdriver.Chrome(r"chromedriver.exe")

# Opening flipkart page on automated chrome web browser
driver.get("https://www.flipkart.com/")
time.sleep(2)  # wait for 2 seconds to load the page

# closing signin page
close_signin=driver.find_element(By.XPATH, '//button[@class="_2KpZ6l _2doB4z"]')
close_signin.click()

# maximize the window's size
driver.maximize_window()

# finding search element in the web page
product=driver.find_element(By.CLASS_NAME, "_3704LK")
# Asking the input from user on the smart phone brand 
brand=input("Enter the smart phone brand name: ")
# entering smart phone brand in search as required
product.send_keys(brand)


# click on search option
search=driver.find_element(By.XPATH, "/html/body/div[1]/div/div[1]/div[1]/div[2]/div[2]/form/div/button")
search.click()
driver.implicitly_wait(3) # seconds

# scrap all product urls
product_urls=[]
start=0
end=1
for page in range(start,end): # for loop for scrapping 1 page
    url=driver.find_elements(By.XPATH,'//div[@class="_13oc-S"]/div/div/a')   # finding url elements all all products on the web page
    for i in url: # for loop for all urls
        product_urls.append(i.get_attribute("href"))  # getting url links of each product on the web page      
    driver.implicitly_wait(2) # seconds

# scrapping details from each url of products
Brand_Name=[]
Smart_Phone_Name=[]
Colour=[]
RAM=[]
Storage_ROM=[]
Camera=[]
Display_Size=[]
Battery_Capacity=[]
Price=[]

for i in product_urls: # for loop for every url of products
    driver.get(i)  # go to url of each product
    driver.implicitly_wait(2) # seconds
    read_more=driver.find_element(By.XPATH,'//button[@class="_2KpZ6l _1FH0tX"]')  # finding element of Read More button
    read_more.click() # click on Read More button
    driver.implicitly_wait(1) # seconds
    Brand_Name.append(brand) # Append brand name details into the list. Here brand is taken from user input
    
    # scrapping smart phone name details
    try:
        phone_name=driver.find_element(By.XPATH,'//div[@class="_1UhVsV"]/div[1]/table/tbody/tr[3]/td[2]/ul/li') # finding elements of phone names
        Smart_Phone_Name.append(phone_name.text) # appending phone names into the list
    except NoSuchElementException:  # Handling exception
        Smart_Phone_Name.append("_") # Use "_" for missing values
        
    # scrapping colour details
    try:
        phone_color=driver.find_element(By.XPATH,'//div[@class="_1UhVsV"]/div[1]/table/tbody/tr[4]/td[2]/ul/li')
        Colour.append(phone_color.text)
    except NoSuchElementException:
        Colour.append("_")
        
    # scrapping RAM details
    try:
        phone_RAM=driver.find_element(By.XPATH,'//div[@class="_1UhVsV"]/div[4]/table/tbody/tr[2]/td[2]/ul/li')
        RAM.append(phone_RAM.text)
    except NoSuchElementException:
        RAM.append("_")
    
    # scrapping storage ROM details
    try:
        phone_ROM=driver.find_element(By.XPATH,'//div[@class="_1UhVsV"]/div[4]/table/tbody/tr[1]/td[2]/ul/li')
        Storage_ROM.append(phone_ROM.text)
    except NoSuchElementException:
        Storage_ROM.append("_")
    
    # scrapping camera details
    try:
        camera_details=driver.find_element(By.XPATH,'/html/body/div[1]/div/div[3]/div[1]/div[2]/div[7]/div[1]/div/div[2]/ul/li[3]')
        Camera.append(camera_details.text)
    except NoSuchElementException:  # exception handling for brand name
        Camera.append("_")  # replace with "_" for missing details
    
    # scrapping display details
    try:
        phone_display_size=driver.find_element(By.XPATH,'//div[@class="_1UhVsV"]/div[2]/table/tbody/tr[1]/td[2]/ul/li')
        Display_Size.append(phone_display_size.text)
    except NoSuchElementException:
        Display_Size.append("_")
    
    # scrapping battery details
    try:
        phone_battery=driver.find_element(By.XPATH,'//li[@class="_21lJbe" and contains(text(),"mAh")]')
        Battery_Capacity.append(phone_battery.text)
    except NoSuchElementException:
        Battery_Capacity.append("_")
    
    # scrapping price details
    try:
        phone_price=driver.find_element(By.XPATH,'//div[@class="dyC4hf"]/div[1]/div/div')
        Price.append(phone_price.text)
    except NoSuchElementException:
        Price.append("_")
        
driver.close()   # closing the driver
        
# printing the lengths of all scrapped items
print(len(Brand_Name),len(Smart_Phone_Name),len(Colour),len(RAM),len(Storage_ROM),len(Camera),len(Display_Size),len(Battery_Capacity),len(Price),len(product_urls))

# creating dataframe
df=pd.DataFrame({'Brand Name':Brand_Name,'Smart Phone Name':Smart_Phone_Name,'Colour':Colour,'RAM':RAM,'Storage ROM':Storage_ROM,'Camera':Camera,'Display Size':Display_Size,'Battery Capacity':Battery_Capacity,'Price':Price,'Product URL':product_urls})

# writing the data to csv file
df.to_csv("question4-df.csv")

# Displaying the dataframe
df

Enter the smart phone brand name: Oneplus Nord
24 24 24 24 24 24 24 24 24 24


,Brand Name,Smart Phone Name,Colour,RAM,Storage ROM,Camera,Display Size,Battery Capacity,Price,Product URL
0,Oneplus Nord,Nord CE 2 Lite,Black Dusk,6 GB,128 MB,_,16.74 cm (6.59 inch),5000 mAh,"₹19,887",https://www.flipkart.com/oneplus-nord-ce-2-lit...
1,Oneplus Nord,Nord CE 2 Lite 5G,Black Dusk,6 GB,128 GB,_,16.74 cm (6.59 inch),5000 mAh,"₹19,800",https://www.flipkart.com/oneplus-nord-ce-2-lit...
2,Oneplus Nord,Nord CE 2 Lite 5G,Blue Tide,6 GB,128 GB,_,16.74 cm (6.59 inch),5000 mAh,"₹19,949",https://www.flipkart.com/oneplus-nord-ce-2-lit...
3,Oneplus Nord,Nord CE 2 Lite 5G,Black Dusk,6 GB,128 GB,_,16.74 cm (6.59 inch),5000 mAh,"₹19,890",https://www.flipkart.com/oneplus-nord-ce-2-lit...
4,Oneplus Nord,Nord CE 2 Lite 5G,Blue Tide,6 GB,128 GB,64MP Rear Camera | 16MP Front Camera,16.74 cm (6.59 inch),5000 mAh,"₹19,999",https://www.flipkart.com/oneplus-nord-ce-2-lit...
5,Oneplus Nord,Nord CE 2 5G,Gray Mirror,8 GB,128 GB,_,16.33 cm (6.43 inch),4500 mAh,"₹24,990",https://www.flipkart.com/oneplus-nord-ce-2-5g-...
6,Oneplus Nord,Nord 2T 5G,Jade Fog,8 GB,128 GB,_,17.02 cm (6.7 inch),4500 mAh,"₹28,990",https://www.flipkart.com/oneplus-nord-2t-5g-ja...
7,Oneplus Nord,Nord CE 2 5G,Bahama Blue,8 GB,128 GB,64MP Rear Camera,16.33 cm (6.43 inch),4500 mAh,"₹24,998",https://www.flipkart.com/oneplus-nord-ce-2-5g-...
8,Oneplus Nord,Nord CE 5G,Charcoal Ink,6 GB,128 GB,_,16.33 cm (6.43 inch),4500 mAh,"₹20,899",https://www.flipkart.com/oneplus-nord-ce-5g-ch...
9,Oneplus Nord,Nord 2T 5G,Jade Fog,12 GB,256 GB,_,17.02 cm (6.7 inch),4500 mAh,"₹33,999",https://www.flipkart.com/oneplus-nord-2t-5g-ja...


# QUESTION 5

In [9]:
# Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

# Connect to the driver
driver=webdriver.Chrome(r"chromedriver.exe")

# Opening google maps page on automated chrome web browser
driver.get("https://www.google.co.in/maps/")
time.sleep(2)

# maximize the window's size
driver.maximize_window()

# finding search element from the web page
product=driver.find_element(By.ID, "searchboxinput")
# Asking user inputs on city name to search geospatial coordinates
city=input("Enter the city name: ")
# sending city name in search
product.send_keys(city)

# click on search option
search=driver.find_element(By.ID, "searchbox-searchbutton")
search.click()
driver.implicitly_wait(3) # seconds

# scrapping geospatial coordinates of city
try:
    time.sleep(5) # wait for 5 seconds to go to next line of code
    url_string=driver.current_url  # getting current web page url text
    print("URL Extracted:",url_string)  # printing the URL text extracted
    lat_lng=re.findall(r'@(.*),11z',url_string) # Using regular expression to get the data between "@" and ",11z" from the url string
    result=lat_lng[0].split(",")  # Both coordinates are saved as single item in the list hence splitting the first list item to seperate both coordinates 
except NoSuchElementException: # Error Handling exception
    print("Error")  
index_column=["{} Latitude".format(city),"{} Longitude".format(city)]  # defining index column with names of Longitude and Latitude
df=pd.DataFrame({"Geospatial Coordinates":result,"Index":index_column}) # creating data frame with both coordinates and index column
df.set_index("Index",inplace=True) # chaning the index column in the data frame to see the text in the first column

driver.close()   # closing the driver

df # displaying the dataframe

Enter the city name: Bengaluru
URL Extracted: https://www.google.co.in/maps/place/Bengaluru,+Karnataka/@12.95396,77.4908519,11z/data=!3m1!4b1!4m5!3m4!1s0x3bae1670c9b44e6d:0xf8dfc3e8517e4fe0!8m2!3d12.9715987!4d77.5945627


,Geospatial Coordinates
Index,
Bengaluru Latitude,12.95396
Bengaluru Longitude,77.4908519


# QUESTION 6

In [11]:
# Write a program to scrap details of all the funding deals for first quarter (i.e Jan 21 – March 21) from trak.in.

web_page = "https://trak.in/india-startup-funding-investment-2015/"
Info = requests.get(web_page).text
bsoup=BeautifulSoup(Info, "lxml")  # using beautiful soup to extract the data
rows = bsoup.find_all('tr')
contents = []
for i in rows:
    columns=i.find_all('td')
    columns=[x.text.strip() for x in columns]
    contents.append(columns)
# creating the dataframe with contents
df = pd.DataFrame(data=contents)
# Defining all column names
df.columns =['S No','Date', 'StartUp Name', 'Industry', 'Subvertical','City','Investors name','Investment Type','Amount','']
df = df.iloc[1: , :-1]  # extracting the required data
Index_Jan=(df[df['Date']=='11/01/2021'].index.values) # Get the row number where the date matches with "11/01/2021"
Index_March=(df[df['Date']=='07/04/2021'].index.values) # Get the row number where the date matches with "07/04/2021"
df_new1=df.dropna(axis=0, how='any')  # dropping rows with null values if any present in all the rows
df_new2=df_new1.iloc[Index_March[0]+1:Index_Jan[0]-2]  # Final dataframe for first quarter (i.e. Jan 21 to March 21)
df_new2 # displaying dataframe

,S No,Date,StartUp Name,Industry,Subvertical,City,Investors name,Investment Type,Amount
7,1,04/03/2021,DealShare,E-commerce,Online shopping platform,"Jaipur, Rajasthan",Innoven Capital,Debt Financing,"250,000,000"
8,2,31/03/2021,Uniphore,Technology,Conversational Service Automation (CSA),Palo Alto,Sorenson Capital Partners,Series D,"140,000,000"
9,3,30/03/2021,Dunzo,E-commerce,Hyper-local delivery app,Bengaluru,Krishtal Advisors Pte Ltd,Series E,"8,000,000"
10,4,30/03/2021,BYJU’S,Edu-tech,Online tutoring,Bengaluru,"MC Global Edtech, B Capital, Baron, others",Series F,"460,000,000"
11,5,23/03/2021,SkilloVilla,Edu-tech,Career and job-oriented upskilling.,Bengaluru,"Titan Capital, others",Seed,"300,000,000"
12,6,25/03/2021,CityMall,E-commerce,Social ecommerce and online grocery platform,Gurgaon,Accel Partners,Series A,"11,000,000"
13,7,26/03/2021,DotPe,FinTech,Commerce and payments platform to offline ente...,Gurgaon,PayU,Series A,"27,500,000"
15,1,11/02/2021,Doubtnut,Edu Tech,E-Learning Platform,Gurgaon,"SIG Global, Sequoia Capital, WaterBridge Ventu...",Series B,"2,500,000"
16,2,22/02/2021,Zomato,Hospitality,Online Food Delivery Platform,Gurgaon,"Tiger Global, Kora",Venture,"250,000,000"
17,3,19/02/2021,Fingerlix,Hospitality,Semi-cooked food delivery app,Mumbai,"Rhodium Trust, Accel Partners and Swiggy",Series C,"2,747,045.20"


# QUESTION 7

In [12]:
# Write a program to scrap all the available details of best gaming laptops from digit.in.

# Connect to the driver
driver=webdriver.Chrome(r"chromedriver.exe")

# Opening https://www.digit.in/ page on automated chrome web browser
driver.get("https://www.digit.in/")
time.sleep(3)

# maximize the window's size
driver.maximize_window()

# click on TOP10 option on the web page
top=driver.find_element(By.XPATH,'/html/body/div[1]/div/div[4]/ul/li[4]/a')
top.click()
time.sleep(3)

# click on LAPTOPS option in TOP10 of the web page
laptops=driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[2]/div[5]/div[1]/div/button[2]')
laptops.click()
time.sleep(3)

# click on Best Gaming Laptops in LAPTOPS
gaming_laptops=driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[2]/div[5]/div[3]/div[3]/a/div[2]/p')
gaming_laptops.click()
time.sleep(3)

# scrapping details from each url of products
Name=[]
OS=[]
Display=[]
Processor=[]
Memory=[]
Graphics_Processor=[]
Body=[]
Price=[]
Seller=[]

# scrapping all details

# Scrapping laptop names
laptop_name=driver.find_elements(By.XPATH, '//div[@class="specfpro_details_cont"]/div/div[1]/a/h3') 
try:
    for i in laptop_name:       
        Name.append(i.text)
except NoSuchElementException:
    Name.append("_")

# scrapping laptop OS data    
laptop_os=driver.find_elements(By.XPATH, '//div[@class="Spcs-details"]/table/tbody/tr[5]/td[3]')
try:
    for i in laptop_os:       
        OS.append(i.text)
except NoSuchElementException:
    OS.append("_")

# scrapping display details    
laptop_display=driver.find_elements(By.XPATH, '//div[@class="Spcs-details"]/table/tbody/tr[4]/td[3]')
try:
    for i in laptop_display:       
        Display.append(i.text)
except NoSuchElementException:
    Display.append("_")
    
# scrapping processor details
laptop_processor=driver.find_elements(By.XPATH, '//div[@class="Spcs-details"]/table/tbody/tr[3]/td[3]')
try:
    for i in laptop_processor:       
        Processor.append(i.text)
except NoSuchElementException:
    Processor.append("_")

# scrapping laptop memory details    
laptop_memory=driver.find_elements(By.XPATH, '//div[@class="Spcs-details"]/table/tbody/tr[6]/td[3]')  
try:
    for i in laptop_memory:       
        Memory.append(i.text)
except NoSuchElementException:
    Memory.append("_")

# scrapping graphics processor details    
graphics_processor=driver.find_elements(By.XPATH, '//div[@class="Spcs-details"]/table/tbody/tr[7]/td[3]') 
try:
    for i in graphics_processor:       
        Graphics_Processor.append(i.text)
except NoSuchElementException:
    Graphics_Processor.append("_")

# scrapping laptop body details    
laptop_body=driver.find_elements(By.XPATH, '//div[@class="Spcs-details"]/table/tbody/tr[8]/td[3]') 
try:
    for i in laptop_body:       
        Body.append(i.text)
except NoSuchElementException:
    Body.append("_")

# scrapping laptop seller details    
laptop_seller=driver.find_elements(By.XPATH, '//td[@class="smmerchant"]')
try:
    for i in laptop_seller:       
        Seller.append(i.text)
except NoSuchElementException:
    Seller.append("_")

# scrapping laptop price details    
# laptop_price=driver.find_elements(By.XPATH, '//div[@class="Spcs-details"]/table/tbody/tr[9]/td[3]')
laptop_price=driver.find_elements(By.XPATH, '//td[@class="smprice"]')
try:
    for i in laptop_price:       
        Price.append(i.text)
except NoSuchElementException:
    Price.append("_")

driver.close()   # closing the driver    
    
# printing lengths of all scrapped items
print(len(Name),len(OS),len(Display),len(Processor),len(Memory),len(Graphics_Processor),len(Body),len(Seller),len(Price))

# creating a dataframe
df=pd.DataFrame({'Laptop Name':Name,'OS':OS,'Display':Display,'Processor':Processor,'Memory':Memory,'Graphics Processor':Graphics_Processor,'Body':Body,'Seller':Seller,'Price':Price})
df # displaying the data frame

10 10 10 10 10 10 10 10 10


,Laptop Name,OS,Display,Processor,Memory,Graphics Processor,Body,Seller,Price
0,MSI Titan GT77-12UHS,Windows 11 Home,"17.3″ (3840 x 2160) screen, 120 Hz refresh rate",12th Gen Intel Core i9-12900HX 16 core processor,64 GB DDR5 RAM & 2 TB SSD,16 GB DDR6 NVIDIA GeForce RTX 3080 Ti Graphics...,397 x 330 x 23 mm dimension & 3.3 kg weight,N/A,N/A
1,Alienware X17 R2,Windows 11 Home,"17.3″ (1920 x 1080) screen, 360 Hz refresh rate",12th Gen Intel Core i9-12900H 14 core processo...,32 GB DDR5 RAM & 1 TB SSD,16 GB DDR6 NVIDIA GEFORCE RTX 3080 Ti Graphics...,299 x 359 x 21 mm dimension & 2.96 kg weight,Croma,"₹ 389,990"
2,Acer Predator Triton 500 SE PT516-52s,Windows 11,16″ (2560 x 1600) screen,12th Gen Intel Core i7-12700H 14 core processo...,32 GB DDR5 RAM & 2 TB SSD,8 GB DDR6 NVIDIA GeForce RTX 3070 Ti Graphics ...,262 x 359 x 20 mm dimension & 2.4 kg weight,Amazon,"₹ 229,990"
3,Omen By HP (16-B1371TX),Windows 11 Home,16.1″ (2560 x 1440) screen,12th Gen Intel Core i7-12700H 14 core processo...,8 GB DDR5 RAM & 1 TB SSD,8 GB GDDR6 NVIDIA GeForce RTX 3070 Graphics card,369 x 248 x 23 mm dimension & 2.32 kg weight,Amazon,"₹ 176,490"
4,Acer Predator Helios 300 AN515-45,Windows 11 Home,"15.6″ (2560 x 1440) screen, 165 Hz refresh rate",AMD Ryzen 9-5900HX 8 core processor,16 GB DDR4 RAM & 512 GB SSD,8 GB DDR6 NVIDIA GeForce RTX 3070 Graphics card,363 x 255 x 23.9 mm dimension & 2.4 kg weight,N/A,"₹ 172,999"
5,MSI Delta 15 (A5EFK-083IN),Windows 11 Home,"15.6″ (1920 x 1080) screen, 240Hz refresh rate",AMD 5th Gen Ryzen 9-5900HX 8 core processor wi...,16 GB DDR4 RAM & 1 TB SSD,10 GB DDR6 AMD Radeon RX 6700M Graphics card,357 x 247 x 19 mm dimension & 1.9 kg weight,N/A,"₹ 188,990"
6,Omen By HP (16-C0141AX),Windows 11 Home,16.1″ (2560 x 1440) screen,AMD Ryzen™ 9 5900HX 8 core processor with 3.3 ...,16 GB DDR4 RAM & 1 TB NVMe,8 GB GDDR6 AMD Radeon™ RX 6600M Graphics card,36.92 x 24.8 x 2.3 mm dimension & 2.3 kg weight,Croma,"₹ 124,990"
7,Lenovo Legion 5i Pro (82RF00MGIN),Windows 11 Home,"16″ (2560 x 1600) screen, 165 Hz refresh rate",12th Gen Intel Core i7-12700H 14 core processo...,16 GB DDR5 RAM & 1 TB SSD,6 GB DDR6 NVIDIA GeForce RTX 3060 Graphics card,360 x 264 x 20 mm dimension & 2.49 kg weight,Amazon,"₹ 169,990"
8,Alienware M15 R5 Ryzen Edition Icc-C780001win,Windows 11 Home,"15.6″ (1920 x 1080) screen, 165Hz refresh rate",AMD Ryzen R7-5800H 8 core processor with 4.40 ...,16 GB DDR4 RAM & 512 GB SSD,6 GB DDR6 NVIDIA GeForce RTX 3060 Graphics card,356.2 x 272.5 x 22.85 mm dimension & 2.69 kg w...,Amazon,"₹ 174,000"
9,Lenovo Slim 7 Gen 6 (82K8002JIN),Windows 11 Home,"15.6″ screen, 165 Hz refresh rate",Lenovo Ryzen 7-5800H processor with 3.2 GHz cl...,16 GB DDR4 RAM & 1 TB SSD,6 GB DDR6 NVIDIA GeForce 3060 Max-Q Graphics card,356 x 252 x 16 mm dimension & 1.9 kg weight,Croma,"₹ 131,990"


# QUESTION 8

In [13]:
# Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, 
#“Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

# Connect to the driver
driver=webdriver.Chrome(r"chromedriver.exe")

# Opening www.forbes.com page on automated chrome web browser
driver.get("https://www.forbes.com/")
time.sleep(5)

# maximize the window's size
driver.maximize_window()

# click on LEFT MOST option
top=driver.find_element(By.XPATH,'/html/body/div[1]/header/nav/div[1]/div/div/div[1]')
top.click()
time.sleep(5)

# click on Billionaires option
laptops=driver.find_element(By.XPATH,'/html/body/div[1]/header/nav/div[1]/div/div/div[2]/ul/li[1]/div[1]')
laptops.click()
time.sleep(5)

# click on top 10 Billionaires
gaming_laptops=driver.find_element(By.XPATH,'/html/body/div[1]/main/div/section/section[1]/div/div/div[1]/div/div[1]/div[1]/div[2]/a/h2')
gaming_laptops.click()
driver.implicitly_wait(20) # wait the driver for 20 seconds

for _ in range(3): # scroll the page for 5 times
    time.sleep(2) # wait for 2 seconds to go to next command 
    driver.execute_script("window.scrollBy(0,500)")  # scroll the page by 500 pixes each time

# scrapping details from each url of products
Rank=[]
Name=[]
Net_Worth=[]
Age=[]
Citizenship=[]
Source=[]
Industry=[]

# scrapping all details
time.sleep(20) # wait for 20 seconds to go to next command 
# scrapping rank data
Rank_data=driver.find_elements(By.XPATH, '//div[@class="rank"]') 
try:
    for i in Rank_data:       
        Rank.append(i.text)
except NoSuchElementException:
    Rank.append("_")

# scrapping name data    
Name_data=driver.find_elements(By.XPATH, '//div[@class="personName"]') 
try:
    for i in Name_data:       
        Name.append(i.text)
except NoSuchElementException:
    Name.append("_")

# scrapping net worth data    
Net=driver.find_elements(By.XPATH, '//div[@class="netWorth"]') 
try:
    for i in Net:       
        Net_Worth.append(i.text)
except NoSuchElementException:
    Net_Worth.append("_")

# scrapping age data    
age_data=driver.find_elements(By.XPATH, '//div[@class="age"]') 
try:
    for i in age_data:       
        Age.append(i.text)
except NoSuchElementException:
    Age.append("_")

# scrapping citizenship data    
Citizenship_data=driver.find_elements(By.XPATH, '//div[@class="countryOfCitizenship"]') 
try:
    for i in Citizenship_data:       
        Citizenship.append(i.text)
except NoSuchElementException:
    Citizenship.append("_")

# scrapping source data    
Source_data=driver.find_elements(By.XPATH, '//span[@class="source-text"]') 
try:
    for i in Source_data:       
        Source.append(i.text)
except NoSuchElementException:
    Source.append("_")

# scrapping industry data    
Industry_data=driver.find_elements(By.XPATH, '//div[@class="category"]') 
try:
    for i in Industry_data:       
        Industry.append(i.text)
except NoSuchElementException:
    Industry.append("_")
    
driver.close()   # closing the driver    

# priting lengths of all scrapped items    
print(len(Rank),len(Name),len(Net_Worth),len(Age),len(Citizenship),len(Source),len(Industry))

# creating a dataframe with all the scrapped details
df=pd.DataFrame({'RANK':Rank,'NAME':Name,'NET WORTH':Net_Worth,'AGE':Age,'COUNTRY':Citizenship,'SOURCE':Source,'INDUSTRY':Industry})

# display dataframe
df

200 200 200 200 200 200 200


,RANK,NAME,NET WORTH,AGE,COUNTRY,SOURCE,INDUSTRY
0,1.,Elon Musk,$219 B,50,United States,"Tesla, SpaceX",Automotive
1,2.,Jeff Bezos,$171 B,58,United States,Amazon,Technology
2,3.,Bernard Arnault & family,$158 B,73,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$129 B,66,United States,Microsoft,Technology
4,5.,Warren Buffett,$118 B,91,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...
195,192.,Marcel Herrmann Telles,$10.3 B,72,Brazil,beer,Food & Beverage
196,197.,Leon Black,$10 B,70,United States,private equity,Finance & Investments
197,197.,Joe Gebbia,$10 B,40,United States,Airbnb,Technology
198,197.,David Geffen,$10 B,79,United States,"movies, record labels",Media & Entertainment


# QUESTION 9

In [15]:
# Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

# Connect to the driver
driver=webdriver.Chrome(r"chromedriver.exe")

# Opening youtube video page on automated chrome web browser
video_link=input("Enter YouTube Video Link: ")  # Asking the user the Youtube video link
driver.get(video_link) # Open youtube video in the web browser
# driver.get("https://www.youtube.com/watch?v=gH_RYRwVrVM")
time.sleep(5) # sleep for 5 seconds

# Below three lines written to load the page initially slowly with fewer pixels. This is written because webpage is not loading comments if directly scrolling with more pixles and without waiting time. This is just to load initial comments.
for _ in range(5):  # scroll the page 5 times
    time.sleep(1) # wait for 1 second to go to next command  
    driver.execute_script("window.scrollBy(0,100)") # scrolll the web page to load 100 pixels each time
        
# Below two lines written to load the page with more comments to scrap
for _ in range(3000):  # scroll the page 3000 times. This is needed to load 500 comments
    driver.execute_script("window.scrollBy(0,10000)")   # scrolling the page to load 10,000 pixles each time

# maximize the window's size
driver.maximize_window()

time.sleep(5) # wait for 5 seconds to execute next set of code lines

# defining empty list items as required
Comments=[]
Comment_UpVote=[]
Comment_Posted_Time=[]

# scrapping all details

# scrapping video comments  
video_comments=driver.find_elements(By.ID, 'content-text') # finding all elements of video comments
try:
    for i in video_comments: # for loop with no. of video comments
        if video_comments.index(i)>499: # checking if the no. of video comments is > 500 or not
            break  # break the loop if no. of comments > 500 
        Comments.append(i.text) # appending the video comments to the list
except NoSuchElementException: # Error handling exception
    Comments.append("_")  # Use "_" for missing comments
    
# scrapping the number of comment likes 
comment_likes=driver.find_elements(By.XPATH,'//span[@id="vote-count-middle"]') # finding all elements of comment likes
try:
    for i in comment_likes: 
        if comment_likes.index(i)>499:
            break
        Comment_UpVote.append(i.text)
except NoSuchElementException:
    Comment_UpVote.append("_")
    
# scrapping the comment post details
Time_Post=driver.find_elements(By.XPATH,'//yt-formatted-string[@class="published-time-text style-scope ytd-comment-renderer"]')
try:
    for i in Time_Post: 
        if Time_Post.index(i)>499:
            break
        Comment_Posted_Time.append(i.text)
except NoSuchElementException:
    Comment_Posted_Time.append("_")
    
driver.close() # closing the driver    

# printing lenghts of all scrapped items     
print(len(Comments),len(Comment_UpVote),len(Comment_Posted_Time))

# creating the dataframe with scrapped data
df=pd.DataFrame({'Comments':Comments,'Comment UpVote':Comment_UpVote,'Comment Posted Time':Comment_Posted_Time})

# saving the dataframe data to csv file
df.to_csv("youtube_data.csv")

# displaying the dataframe
df

Enter YouTube Video Link: https://www.youtube.com/watch?v=gH_RYRwVrVM
500 500 500


,Comments,Comment UpVote,Comment Posted Time
0,Great culture Great people kannadigas.....- A ...,25K,2 months ago
1,Divided by language\nUnited by culture\nThank ...,2.5K,2 weeks ago
2,Who else thinks Rishabh Shetty should get Nati...,794,2 weeks ago
3,I am Gujarati. Learned a lot about this cultur...,601,2 weeks ago
4,Proud to say I belong to India of such great c...,376,9 days ago
...,...,...,...
495,"And the ""NATIONAL AWARD FOR THE BEST MOVIE"" OF...",3,6 days ago
496,Start the song and realise 4 mins 31 seconds h...,1,6 days ago
497,I get chills and feel peace at the same time..,,18 hours ago
498,Divine blockbuster...true that,,5 days ago


# QUESTION 10

In [17]:
# Connect to the driver
driver=webdriver.Chrome(r"chromedriver.exe")

# maximize the window's size
driver.maximize_window()

# Opening web page on automated chrome web browser
driver.get("https://www.hostelworld.com")
driver.implicitly_wait(10) # seconds

search=driver.find_element(By.XPATH,'//input[@class="location-text"]') # Finding Search location
search.click() # Clikcing on Search location
search.send_keys("London") # Sending "London" in search location
time.sleep(5) # wait for 5 seconds to go to next line of code
search.clear() # Clearing the search field as it is not populating Core city when entering "London" first time in the search location
driver.find_element(By.XPATH,'//input[@class="location-text"]').send_keys("London") # Finding and enter "London" again in search box
time.sleep(5) # wait for 5 seconds to load the core city
driver.find_element(By.XPATH,'//i[@class="core-icon icon-core-city"]').click() # Finding and clicking on core city symbol
driver.find_element(By.XPATH,'//div[@class="search-button"]').click()  # Finding and Clicking on "Let's go"
time.sleep(5) # wait for 5 seconds to go to next line of code 
driver.execute_script("window.scrollBy(0,500)") #scroll the page by 500 pixels to load the page
time.sleep(5) # wait for 5 seconds to go to next line of code 

# defining empty list items as required
Hostel_Name=[]
Distance_from_city_centre=[]
Ratings=[]
Total_reviews=[]
Privates_from_price_i=[]
Privates_from_price_f=[]
Dorms_from_price_i=[]
Dorms_from_price_f=[]
Facilities=[]
Property_description=[]

start=0
end=2
for page in range(start,end): # for loop for scrapping 3 pages
    # scrapping Hostel Names
    name=driver.find_elements(By.XPATH, '//h2[@class="title title-6"]') # finding all elements of TITLES
    try:
        for i in name: # for loop with no. of hostel names
            Hostel_Name.append(i.text) # appending the hostel names to the list
    except NoSuchElementException: # Error handling exception
        Hostel_Name.append("_")  # Use "_" for missing names
    
    # scrapping Distance from city centre
    name=driver.find_elements(By.XPATH, '//span[@class="description"]') 
    try:
        for i in name: 
            Distance_from_city_centre.append(i.text)
    except NoSuchElementException: 
        Distance_from_city_centre.append("_")  
    
    # scrapping Ratings
    rating_info = driver.find_elements(By.XPATH, "//div[@class='bottom-rating']//div[@class='rating rating-summary-container big']//span" or "//a[@class='property-card-container horizontal']//div[@class='property-rating']//div[@class='score-row']")
    try:
        for i in rating_info: 
            Ratings.append(i.text) 
    except NoSuchElementException: 
        Ratings.append("_") 
        
    # scrapping Total Reviews
    reviews=driver.find_elements(By.XPATH, '//div[@class="reviews"]') 
    try:
        for i in reviews: 
            Total_reviews.append(i.text) 
    except NoSuchElementException: 
        Total_reviews.append("_")  
        
    # scrapping Privates_from_price
    privates_price=driver.find_elements(By.XPATH,'//div[@class="price-col"][1]') 
    try:
        for i in privates_price: 
            Privates_from_price_i.append(i.text) 
    except NoSuchElementException: 
        Privates_from_price_i.append("_")  
   
    # scrapping Dorms_from_price
    dorms_price=driver.find_elements(By.XPATH,'//div[@class="price-col"][2]')
    try:
        for i in dorms_price: 
            Dorms_from_price_i.append(i.text) 
    except NoSuchElementException: 
        Dorms_from_price_i.append("_")  
        
    # scrapping Facilities
    facilities_info=driver.find_elements(By.XPATH,'//div[@class="facilities-label facilities"]')
    try:
        for i in facilities_info: 
            Facilities.append(i.text)
    except NoSuchElementException: 
        Facilities.append("_")  
    
    driver.execute_script("window.scrollBy(0,9000)")  # scroll the page by 9000 pixels
    time.sleep(5)  # wait for 5 seconds to go to next line of code
    # Finding and Clicking on next button to go to next page
    if (page>=1):
        break
    else:
        next_button=driver.find_element(By.XPATH, "//i[@class='core-icon icon-core-chevron-right']")
        next_button.click()
        time.sleep(5)  # wait for 5 seconds to go to next line of code
                        
# for loop to segregage private price information from initial scrapped data   
for item1 in Privates_from_price_i:
    result1=item1.split("\n")
    result2=[]
    for item2 in result1:
        if(item2!="Privates From"):
            result2.append(item2)
    result3=" ".join(result2)
    Privates_from_price_f.append(result3)

# for loop to segregage dorms price information from initial scrapped data 
for item1 in Dorms_from_price_i:
    result1=item1.split("\n")
    result2=[]
    for item2 in result1:
        if(item2!="Dorms From"):
            result2.append(item2)
    result3=" ".join(result2)
    Dorms_from_price_f.append(result3)

driver.close() # closing the driver 

# printing lengths of all items    
print(len(Hostel_Name),len(Distance_from_city_centre),len(Ratings),len(Total_reviews),len(Privates_from_price_f),len(Dorms_from_price_f),len(Facilities))

# creating a data frame with all the information scrapped
df=pd.DataFrame({'Hostel Names':Hostel_Name,'Distance from city centre':Distance_from_city_centre,'Ratings':Ratings,'Total Reviews':Total_reviews,'Privates from price':Privates_from_price_f,'Dorms from price':Dorms_from_price_f,'Facilities':Facilities})

# displaying dataframe
df

55 55 55 55 55 55 55


,Hostel Names,Distance from city centre,Ratings,Total Reviews,Privates from price,Dorms from price,Facilities
0,Wombat's City Hostel London,Hostel - 3.6km from city centre,Fabulous,14420 Total Reviews,Rs15953.5 Rs12763,Rs3663.42 Rs2931,Free WiFi\nFollows Covid-19 sanitation guidance
1,St Christopher's Village,Hostel - 1.8km from city centre,Fabulous,11770 Total Reviews,No Privates Available,Rs1762.62 Rs1586,Free WiFi\nFollows Covid-19 sanitation guidance
2,Hostel One Notting Hill,Hostel - 5.5km from city centre,Superb,1643 Total Reviews,Rs12791,Rs3090,Free WiFi
3,Urbany Hostel London,Hostel - 5.4km from city centre,Superb,577 Total Reviews,Rs9974,Rs2641,Free WiFi\nFollows Covid-19 sanitation guidance
4,Generator London,Hostel - 3km from city centre,Very Good,7297 Total Reviews,Rs6800,Rs2049,Free WiFi\nFollows Covid-19 sanitation guidance
5,Safestay London Elephant & Castle,Hostel - 1.7km from city centre,Very Good,4736 Total Reviews,Rs18873,Rs6620,Free WiFi\nFollows Covid-19 sanitation guidance
6,Smart Russell Square Hostel,Hostel - 2.6km from city centre,Very Good,9848 Total Reviews,Rs10844,Rs1833,Free WiFi\nFollows Covid-19 sanitation guidance
7,No.8 Seven Sisters,Hostel - 9km from city centre,Rating,3930 Total Reviews,No Privates Available,Rs977,Free WiFi\nFollows Covid-19 sanitation guidance
8,No.8 Willesden Hostel London,Hostel - 10km from city centre,Good,4914 Total Reviews,Rs2225,No Dorms Available,Free WiFi\nFollows Covid-19 sanitation guidance
9,Smart Hyde Park Inn Hostel,Hostel - 5km from city centre,Very Good,6452 Total Reviews,No Privates Available,Rs1937.19 Rs1743,Free WiFi
